## Research

**using this there is no need to reload your packages every time when you will make any improvement to them**

In [1]:
%load_ext autoreload

In [2]:
autoreload 2

**import necessary libraries**

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import sys
import warnings

**set warnings options**

In [4]:
warnings.filterwarnings('ignore')

#### to have ease in importing modules

In [5]:
sys.path.insert(0, '/Users/mjasiecz/PycharmProjects/new_offer_success_predictor/src')

**set pandas options**

In [6]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 200

**import packages for manage and preprocess data**

In [7]:
from data.data_manager import DataManager

**load data pipeline**

In [8]:
DM = DataManager()
df = DM.load_data()

#### presentation part, done after modelling

In [16]:
gender_insight = df[['accepted', 'gender']].groupby(['gender', 'accepted']).size().reset_index().rename(columns={0:'size'})

In [21]:
gender_insight[gender_insight['gender'] == 'female']

,gender,accepted,size
0,female,no,127
1,female,yes,331


In [23]:
# hardcoded, only plots purposes

In [24]:
331/(127+331)

0.722707423580786

In [27]:
127+331

458

In [22]:
gender_insight[gender_insight['gender'] == 'male']

,gender,accepted,size
2,male,no,673
3,male,yes,158


In [25]:
158/(158+673)

0.1901323706377858

In [26]:
158+673

831

In [29]:
831/(458+831)

0.6446858029480217

In [45]:
df_cc = df[['customer_code', 'gender', 'accepted']]

In [54]:
duplicats_cc = df_cc[df_cc.duplicated(subset=['customer_code'], keep=False)]

In [56]:
duplicats_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, C7CBB5C5613449B to 2E67CB8AC1D8443
Data columns (total 3 columns):
customer_code    576 non-null object
gender           600 non-null object
accepted         600 non-null object
dtypes: object(3)
memory usage: 18.8+ KB


In [59]:
duplicats_cc = duplicats_cc[duplicats_cc['customer_code'].notna()]

In [60]:
duplicats_cc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 576 entries, C7CBB5C5613449B to 2E67CB8AC1D8443
Data columns (total 3 columns):
customer_code    576 non-null object
gender           576 non-null object
accepted         576 non-null object
dtypes: object(3)
memory usage: 18.0+ KB


In [79]:
duplicats_cc['customer_code'].nunique()

210

In [64]:
# add customer_code response ratio as a feature 

In [68]:
len(df)-len(duplicats_cc)

713

In [102]:
713/1289

0.5531419705197828

In [73]:
behavior =        (duplicats_cc
         .sort_values('customer_code')
         .groupby(['customer_code', 'accepted'])
         .size()
         .reset_index()
         .rename(columns={0: 'size'}))

In [81]:
identical_response= behavior[~behavior.duplicated(subset=['customer_code'], keep=False)]

In [83]:
len(identical_response)

128

In [87]:
128/210

0.6095238095238096

In [85]:
identical_response['size'].sum()

345

In [86]:
345/576

0.5989583333333334

In [100]:
duplicats_cc[['customer_code']].groupby(['customer_code']).size().reset_index().groupby([0]).size()

0
2     129
3      49
4      13
5       7
6       4
7       7
11      1
dtype: int64

In [101]:
129*2+3*49+4*13+5*7+6*4+7*7+11

576

In [107]:
duplicats_cc[duplicats_cc['customer_code'] == 'CA. 2343']

,customer_code,gender,accepted
name,,,
7AF493A2CD8E4FA,CA. 2343,male,no
3A9C52F8A32046F,CA. 2343,male,no
881815EF2A8A484,CA. 2343,female,no
B551CD3E07904F1,CA. 2343,female,no
BDF502F66E634B1,CA. 2343,female,no
9044BC863E934F0,CA. 2343,female,no
B02195C26B024EC,CA. 2343,male,no
3FCFB49CFC814CA,CA. 2343,male,no
46D214F8C0684A2,CA. 2343,male,no
